In [5]:
import pandas as pd
pd.set_option('display.max_rows', 5)


In [71]:
# Method to calculate if the team batting second won or not
def is_winner(row):
    if type(row['result']) ==str:
        if row['result'] in row['team_2']:
            return 1
        return 0
    else:
        return 0

In [169]:
# Method to calculate if a player was dismissed on this ball or not
def is_out(row):
    if type(row['player_dismissed']) == str:
        return 1
    else:
        return 0

In [230]:
# Method to calculate if the team batting second won or not for IPL
def ipl_is_winner(row):
    if str(row['batting_team'])== str(row['winner']):
        return 1
    else:
        return 0
    return 0


# Reading Dataset

In [3]:
df = pd.read_csv("./psl_formated.csv")

In [74]:
# Splitting on Innings
first_inning = df[df['inning']==1]
second_inning = df[df['inning']==2]

In [75]:
#Calculating Target
total_sum = first_inning.groupby(["psl_year","match_number"]).agg(
   target= ("runs","sum")
)

In [76]:
total_sum = total_sum.reset_index()

In [78]:
# Merging the DF
new_df = pd.merge(total_sum, second_inning,  how='left', left_on=['psl_year','match_number'], right_on = ['psl_year','match_number'])
                                                                                                          
                                                                                                          

In [80]:
# Calculating if Second Inning team won or not 
new_df['won'] = new_df.apply(is_winner, axis=1)

In [82]:
# Calculating balls left
new_df['balls_left'] = 120-(((new_df['over']-1)*6)+new_df['ball'])

In [83]:
# Creating the Final Dataset
final_df = new_df[['over','ball','total_runs','wickets','target','balls_left','won']]

In [84]:
final_df.head(10)

,over,ball,total_runs,wickets,target,balls_left,won
0,1.0,1.0,0.0,0.0,128,119.0,1
1,1.0,2.0,0.0,0.0,128,118.0,1
2,1.0,3.0,0.0,0.0,128,117.0,1
3,1.0,4.0,2.0,0.0,128,116.0,1
4,1.0,5.0,2.0,0.0,128,115.0,1
5,1.0,6.0,6.0,0.0,128,114.0,1
6,2.0,1.0,7.0,0.0,128,113.0,1
7,2.0,2.0,7.0,0.0,128,112.0,1
8,2.0,3.0,11.0,0.0,128,111.0,1
9,2.0,4.0,12.0,0.0,128,110.0,1


In [87]:
final_df.to_csv("data_without_team_and_match.csv", index=False)

# IPL Data

In [7]:
# Loading Ball by Ball data
ipl_df = pd.read_csv("./Ball-by-ball.csv")

In [245]:
# Loading Match Summary
ipl_match = pd.read_csv("Match-by-match.csv")

In [246]:
ipl_match_winner = ipl_match[['id','winner']]

In [247]:
# Merging Both
ipl_df = pd.merge(ipl_match_winner, ipl_df,  how='left', left_on=['id'], right_on = ['match_id'])
                                                                                                          
                                                                                                          

In [249]:

first_inning =  ipl_df[ipl_df['inning']==1]

In [250]:
# Calculating Target
first_inning_total = first_inning.groupby("match_id").agg(
target = ("total_runs","sum")
)

In [251]:
second_inning = ipl_df[ipl_df['inning']==2]

In [252]:
# Seeing if a person was out on the ball or not
second_inning['out'] = second_inning.apply(is_out, axis=1)

/Users/hassan/opt/anaconda3/envs/tweet/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [254]:
second_inning = second_inning[['match_id','batting_team','over','ball','total_runs','out','winner']]

In [255]:
# Calculating total run on each ball
second_inning['total'] = second_inning.groupby("match_id").agg(
    total=('total_runs','cumsum')
)
#Calculating how many wickets fell
second_inning['wickets'] =second_inning.groupby("match_id").agg(
    total=('out','cumsum')
)

/Users/hassan/opt/anaconda3/envs/tweet/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hassan/opt/anaconda3/envs/tweet/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [259]:
first_inning_total= first_inning_total.reset_index()

In [260]:
# merging DF
merged_df = pd.merge(first_inning_total, second_inning,  how='left', left_on=['match_id'], right_on = ['match_id'])

In [262]:
merged_df['runs_left']= merged_df['target'] -merged_df['total']
merged_df['balls_left'] = 120 -(((merged_df['over']-1)*6)+merged_df['ball'])

In [264]:
merged_df['won'] =merged_df.apply(ipl_is_winner, axis=1)

In [266]:
final_df_ipl = merged_df[['wickets','balls_left','runs_left','won']]

In [270]:
final_df_ipl.shape

(86244, 4)

In [269]:
final_df_ipl.to_csv("ipl_formated.csv",index=False)